In [9]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

In [10]:
### Installation of the necessary packages

#general packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import seaborn
from scipy import stats
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import LabelEncoder

#classifiers
from sklearn import model_selection
from sklearn import metrics
from sklearn import feature_selection
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV


#other imports
from worcliver.load_data import load_data

### Loading the data
data = load_data()

### Division into training and test
amount_in_training = round(len(data.index)*0.8)
amount_in_test = round(len(data.index)*0.2)

#split the data into training data and test data
training_data, test_data = model_selection.train_test_split(data, test_size=(amount_in_test) / len(data), random_state=42)

#checking the lengths
print(f'Training + validation: {len(training_data)} samples, Test: {len(test_data)} samples.')
print(f'So total number of samples: {len(training_data)+len(test_data)}')

### Data inspection: only on the training set!

print(f'The number of samples: {len(training_data.index)}')
print(f'The number of columns: {len(training_data.columns)}')

training_data_df = pd.DataFrame(training_data)
print(training_data_df.info())
cat_cols = training_data_df.select_dtypes(include=["object"]).columns
num_cols = training_data_df.select_dtypes(include=["number"]).columns
print(f"categorical columns: {len(cat_cols)}")
print(f'numerical columns: {len(num_cols)}')

no_variation_samples = 0
for sample in training_data.index:
    data_sample = training_data.drop(columns=['label'])
    if data_sample.loc[sample].max() == data_sample.loc[sample].min():
        no_variation_samples += 1
no_variation_col = 0
for feature in training_data.columns:
    if training_data[feature].max() == training_data[feature].min():
        no_variation_col += 1
print(f'samples with no variation: {no_variation_samples}')
print(f'columns with no variation: {no_variation_col}')

label = training_data[['label']]
malignant_count = (label == "malignant").sum()
benign_count = (label == "benign").sum()

print(f'Class Distribution: malignant {malignant_count} and benign {benign_count}')



Training + validation: 149 samples, Test: 37 samples.
So total number of samples: 186
The number of samples: 149
The number of columns: 494
<class 'pandas.core.frame.DataFrame'>
Index: 149 entries, Liver-030_0 to Liver-103_0
Columns: 494 entries, label to PREDICT_original_phasef_phasesym_entropy_WL3_N5
dtypes: float64(468), int64(25), object(1)
memory usage: 576.2+ KB
None
categorical columns: 1
numerical columns: 493
samples with no variation: 0
columns with no variation: 19
Class Distribution: malignant label    76
dtype: int64 and benign label    73
dtype: int64


## Preprocessing
The function definitions of the functions used for preprocessing.

In [11]:
### Some functions for the preprocessing of the data, decisions made based on training set results

def determining_labels(training_data):
    '''function definition'''
    label_training = training_data[['label']]
    training_data.drop(columns=['label'], inplace=True)

    label_training_binary = label_training.copy()
    label_training_binary['label'] = label_training_binary['label'].map({'malignant': 1, 'benign': 0})
    return label_training_binary

def handling_missing_data(training_data):
    '''function definition'''
    data_training = pd.DataFrame(training_data)

    #if missing data is stored as NaN, replace with the mean
    nan_check = data_training.isna()
    nan_counts = nan_check.sum()
    missing_data_features = nan_counts[nan_counts > 0].index.tolist()

    for feature in missing_data_features:
        mean_value = data_training[feature].mean()
        data_training[feature].fillna(mean_value, inplace=True)
    
    #if missing data is stored as 0, don't handle this as missing data: values in Radiomics can also be determined as 0, so 
    #it cannot be stated that 0 is a missing value. 

    return data_training

def shapiro_wilk_testing(data_training):
    '''function definition'''
    shapiro_results = {}
    no_variation = 0
    features_no_variation = []

    for feature in data_training.columns:
        if data_training[feature].max() == data_training[feature].min():
            shapiro_results[feature] = None  #skipping the feature when there is no variation
            no_variation += 1
            features_no_variation.append(feature)
        else:
            #Shapiro-Wilk test to check for normality
            stat, p_value = stats.shapiro(data_training[feature])

            #if normally distributed, store 1, else store 0
            if p_value > 0.05:
                result = 1
            else:
                result = 0

            shapiro_results[feature] = result

    return features_no_variation, shapiro_results

def normal_distributed_data(shapiro_results): #only relevant on training data, this determines the next steps!
    '''function definition'''
    if sum(1 for result in shapiro_results.values() if result == 0) > 0:
        normal_distributed = False
    return normal_distributed
    
def delete_no_varation(data_training, features_no_variation):
    '''function definition'''
    data_training_clean = data_training.drop(columns=features_no_variation)
    return data_training_clean

def standardize_data(data_training_clean, normal_distributed):
    '''function definition'''
    if not normal_distributed:
        scaler = preprocessing.StandardScaler()
        data_training_standardized = scaler.fit_transform(data_training_clean)
    else:
        data_training_standardized = data_training_clean

    data_training_standardized_df = pd.DataFrame(data_training_standardized, columns=data_training_clean.columns)
    return data_training_standardized_df


## Feature extraction
The function definitions of the functions used for feature extraction.

In [12]:
### Univariate selection

def univariate_feature_selection(data_training_normalized_df):
    '''function definition'''
    corr_matrix = data_training_normalized_df.corr()

    #step 1: calculate the correlation and associated p-value for every feature-pair
    corr_matrix = pd.DataFrame(index=data_training_normalized_df.columns, columns=data_training_normalized_df.columns, dtype=float)
    pval_matrix = pd.DataFrame(index=data_training_normalized_df.columns, columns=data_training_normalized_df.columns, dtype=float)

    for col1 in data_training_normalized_df.columns:
        for col2 in data_training_normalized_df.columns:
            if col1 == col2:
                corr_matrix.loc[col1, col2] = 1.0
                pval_matrix.loc[col1, col2] = 0.0
            else:
                r, p = stats.pearsonr(data_training_normalized_df[col1], data_training_normalized_df[col2])
                corr_matrix.loc[col1, col2] = r
                pval_matrix.loc[col1, col2] = p


    #step 2: loop through all the feature pairs and determine which ones have a high correlation and low p-value
    threshold = 0.9
    p_threshold = 0.05

    to_drop = set()

    for i, col1 in enumerate(data_training_normalized_df.columns):
        for j, col2 in enumerate(data_training_normalized_df.columns):
            if i >= j:
                continue
            r = corr_matrix.loc[col1, col2]
            p = pval_matrix.loc[col1, col2]

            if abs(r) > threshold and p < p_threshold:
                if col1 not in to_drop and col2 not in to_drop:
                    col_to_drop = col2
                    to_drop.add(col_to_drop)

    print("Dropped features due to high correlation:")
    print(to_drop)

    #step 3: drop one of the features in a feature pair with high correlation and low p-value
    data_training_reduced = data_training_normalized_df.drop(columns=to_drop)

    print(f"Size of new training data set is:{data_training_reduced.shape}; Old was: {data_training_normalized_df.shape}")
    return data_training_reduced


def lasso_feature_selection(data_training_reduced, label_training_binary):
    '''function definition'''
    label_training_array = label_training_binary.values.flatten()

    #step 1: perform Lasso with cross-validation for 50 values of alpha on 10 segments over 10.000 iterations
    lasso_cv = LassoCV(alphas=np.logspace(-4, 0, 100), cv=10, max_iter=10000)  
    lasso_cv.fit(data_training_reduced, label_training_array)

    #step 2: find the best alpha value
    best_alpha = lasso_cv.alpha_
    print(f"Optimal Alpha: {best_alpha}")

    #step 3: fit LASSO again with optimal alpha
    lasso_opt = Lasso(alpha=best_alpha)
    lasso_opt.fit(data_training_reduced, label_training_array)

    #step 4: get the features of interest, and only extract those
    selected_features = data_training_reduced.columns[lasso_opt.coef_ != 0]
    print(f"Number of selected features: {len(selected_features)}")
    print(f"Final Selected Features: {list(selected_features)} ")

    training_data_selected = data_training_reduced[selected_features]
    training_data_selected_df = pd.DataFrame(training_data_selected)

    print(f"Size of new training data set is:{training_data_selected.shape}")
    return training_data_selected_df

## Classifiers

### K-Nearest Neighbours
Function definition used for the KNN classifier.

In [ ]:
def KNN_classifier(training_data_selected_df, label_training_binary):

    label_training_array = label_training_binary.values.flatten()
    input_knn = training_data_selected_df

    x_train_KNN = input_knn
    y_train_KNN = label_training_array

    param_grid = {
        'n_neighbors': list(range(1, 31)) ,
    }   

    # Initialize the classifier
    knn = KNeighborsClassifier()

    # GridSearchCV to find the best k
    grid_search = model_selection.GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', n_jobs=-1, verbose=1)

    # Fit model
    grid_search.fit(x_train_KNN, y_train_KNN)

    # # Output results
    # print("Best number of neighbors:", grid_search.best_params_['n_neighbors'])
    # print("Best cross-validation accuracy: {:.2f}".format(grid_search.best_score_))

    # Convert cv_results_ to a DataFrame for easier manipulation
    results_df = pd.DataFrame(grid_search.cv_results_)

    # Vind alle individuele fold score kolommen
    fold_columns = [col for col in results_df.columns if col.startswith("split") and col.endswith("_test_score")]

    # Aantal folds
    n_folds = len(fold_columns)

    # Functie om CI te berekenen per rij
    def compute_confidence_interval(row):
        scores = row[fold_columns].values
        #############################################vanaf hier gaat het mis, scores is leeg
        mean = np.mean(scores)
        std = np.std(scores, ddof=1)  # sample std dev
        ci_halfwidth = stats.norm.ppf(0.975) * std / np.sqrt(n_folds)
        return pd.Series({
            'mean_score': mean,
            'std_score': std,
            'ci_lower': mean - ci_halfwidth,
            'ci_upper': mean + ci_halfwidth
        })

    # Pas toe op elke rij (elke modelconfiguratie)
    ci_df = results_df.apply(compute_confidence_interval, axis=1)
    print(f'ci_df: {ci_df}')

    # Build a DataFrame with all the relevant info
    # final_df = pd.concat([
    #     results_df[['params', 'mean_train_score', 'std_train_score', 'mean_test_score', 'std_test_score']],
    #     ci_df
    # ], axis=1)

    final_df = pd.concat([results_df['params'], ci_df], axis=1)

    # Print de top 5 met hoogste mean_score
    top_models = final_df.sort_values(by='mean_score', ascending=False).head(5)
    print(top_models)

    #retraining the model with the top parameters
    top_params_KNN = top_models.iloc[0]['params']
    final_KNN_model = KNeighborsClassifier(**top_params_KNN)
    final_KNN_model.fit(x_train_KNN, y_train_KNN)

    return final_KNN_model

### Random forest
Function definition used for the random forest classifier.

In [32]:
def random_forest_classifier(training_data_selected_df, label_training_binary):
    label_training_array = label_training_binary.values.flatten()
    input_random_forest = training_data_selected_df
    
    X_train_RF = input_random_forest
    y_train_RF = label_training_array

    param_grid_RF = {
        'n_estimators': [110, 115, 120],                     #trees
        'max_depth': [5, 8, 13],                         #maximum depth
        'bootstrap': [True],                     #bootstrap on or off >> moet dus True of False zijn, maar aangezien er altijd True kwam dat nu even gelaten voor snelheid runnen
        'min_samples_split': [2, 3, 4, 5],                    #minimum samples required to split
        'min_samples_leaf': [1, 2, 3],                  #minimum samples wat je uiteindelijk in een leaf overhoudt 
        'max_features': ['sqrt', 'log2'],                       #controls the number of features selected, can also try log2 
        'min_impurity_decrease': [0.00001, 0.000005, 0.00002]   #controls the minimum amount of impurity reduction required for a split to be created    
    }

    random_forest = RandomForestClassifier(random_state=42)

    #de grid search naar de beste hyperparameters
    grid_search_RF = model_selection.GridSearchCV(random_forest, param_grid_RF, cv=10, scoring='accuracy', n_jobs=-1, verbose=1)
    #grid_search = RandomizedSearchCV(random_forest, param_grid, cv=10, scoring='accuracy', n_iter=20, n_jobs=-1, random_state=42, verbose=1)

    # Fit GridSearchCV to the data
    grid_search_RF.fit(X_train_RF, y_train_RF)

    #find the accuracies per fold
    results_df_RF = pd.DataFrame(grid_search_RF.cv_results_)

    # Vind alle individuele fold score kolommen
    fold_columns_RF = [col for col in results_df_RF.columns if col.startswith("split") and col.endswith("_test_score")]

    # Aantal folds
    n_folds_RF = len(fold_columns_RF)

    # Functie om CI te berekenen per rij
    def compute_confidence_interval_RF(row):
        scores = row[fold_columns_RF].values
        mean = np.mean(scores)
        std = np.std(scores, ddof=1)  # sample std dev
        ci_halfwidth = stats.norm.ppf(0.975) * std / np.sqrt(n_folds_RF)
        return pd.Series({
            'mean_score': mean,
            'std_score': std,
            'ci_lower': mean - ci_halfwidth,
            'ci_upper': mean + ci_halfwidth
        })

    # Pas toe op elke rij (elke modelconfiguratie)
    ci_RF = results_df_RF.apply(compute_confidence_interval_RF, axis=1)

    # Combineer met de parameterinstellingen
    final_RF = pd.concat([results_df_RF['params'], ci_RF], axis=1)

    # Print de top 5 met hoogste mean_score
    top_models_RF = final_RF.sort_values(by='mean_score', ascending=False).head(5)

    pd.set_option('display.max_colwidth', None)
    print(top_models_RF)

    #retraining the model with the top parameters
    top_params_RF = top_models_RF.iloc[0]['params']
    final_RF_model = RandomForestClassifier(**top_params_RF, random_state=42)
    final_RF_model.fit(X_train_RF, y_train_RF)

    return final_RF_model


### Support vector machine
Function definition used for the support vector machine classifier.

In [15]:
def SVM_classifier(training_data_selected_df, label_training_binary):
    label_training_array = label_training_binary.values.flatten()
    input_SVM = training_data_selected_df

    X_train_SVM = input_SVM
    y_train_SVM = label_training_array

    param_grid_SVM = {
        'kernel': ['linear', 'poly', 'rbf'],        # kernel-types
        'C': [0.01, 0.1, 1, 2, 3, 5, 10, 100],      # Expand the range of C values
        'gamma': ['scale', 'auto', 0.01, 0.05, 0.1, 0.2, 0.5], # Add more gamma values
        'degree': [1, 3, 5, 10, 12, 15],            # Add more degree values for polynomial kernel
        'coef0': [0.0, 0.01, 0.5, 1, 2]             # Extend coef0 range
    }

    SVM_model = SVC()

    #de grid search naar de beste hyperparameters
    grid_search_SVM = model_selection.GridSearchCV(SVM_model, param_grid_SVM, cv=10, scoring='accuracy', n_jobs=-1, verbose=1)
    #grid_search = RandomizedSearchCV(random_forest, param_grid, cv=10, scoring='accuracy', n_iter=20, n_jobs=-1, random_state=42, verbose=1)

    # Fit GridSearchCV to the data
    grid_search_SVM.fit(X_train_SVM, y_train_SVM)

    #find the accuracies per fold
    results_df_SVM = pd.DataFrame(grid_search_SVM.cv_results_)

    # Vind alle individuele fold score kolommen
    fold_columns_SVM = [col for col in results_df_SVM.columns if col.startswith("split") and col.endswith("_test_score")]

    # Aantal folds
    n_folds_SVM = len(fold_columns_SVM)

    # Functie om CI te berekenen per rij
    def compute_confidence_interval_SVM(row):
        scores = row[fold_columns_SVM].values
        mean = np.mean(scores)
        std = np.std(scores, ddof=1)  # sample std dev
        ci_halfwidth = stats.norm.ppf(0.975) * std / np.sqrt(n_folds_SVM)
        return pd.Series({
            'mean_score': mean,
            'std_score': std,
            'ci_lower': mean - ci_halfwidth,
            'ci_upper': mean + ci_halfwidth
        })

    # Pas toe op elke rij (elke modelconfiguratie)
    ci_SVM = results_df_SVM.apply(compute_confidence_interval_SVM, axis=1)

    # Combineer met de parameterinstellingen
    final_SVM = pd.concat([results_df_SVM['params'], ci_SVM], axis=1)

    # Print de top 5 met hoogste mean_score
    top_models_SVM = final_SVM.sort_values(by='mean_score', ascending=False).head(5)
    pd.set_option('display.max_colwidth', None)
    print(top_models_SVM)

    #retraining the model with the top parameters
    top_params_SVM = top_models_SVM.iloc[0]['params']
    final_SVM_model = SVC(**top_params_SVM)
    final_SVM_model.fit(X_train_SVM, y_train_SVM)

    return final_SVM_model

### Creating the machine learning models and testing it

In [19]:

### Loading the data
data = load_data()

### Division into training and test
amount_in_training = round(len(data.index)*0.8)
amount_in_test = round(len(data.index)*0.2)

#split the data into training data and test data
training_data, test_data = model_selection.train_test_split(data, test_size=(amount_in_test) / len(data), random_state=42)

### Preprocessing the training data
label_training_binary = determining_labels(training_data)
data_training = handling_missing_data(training_data)
features_no_variation, shapiro_results = shapiro_wilk_testing(data_training)
normal_distributed = normal_distributed_data(shapiro_results)
data_training_clean = delete_no_varation(data_training, features_no_variation)
data_training_normalized_df = standardize_data(data_training_clean, normal_distributed)

### Preprocessing the test data
label_test_binary = determining_labels(test_data)
data_test = handling_missing_data(test_data)
data_test_clean = delete_no_varation(data_test, features_no_variation)
data_test_normalized_df = standardize_data(data_test_clean, normal_distributed)

### Preprocessing all the data together (needed for learning curves)
label_test_binary = determining_labels(data)
data = handling_missing_data(data)
data_clean = delete_no_varation(data, features_no_variation)
data_normalized_df = standardize_data(data_clean, normal_distributed)

### Feature selection
data_training_univariate = univariate_feature_selection(data_training_normalized_df)
training_data_selected_df = lasso_feature_selection(data_training_univariate, label_training_binary)

Dropped features due to high correlation:
{'PREDICT_original_vf_Frangi_edge_peak_SR(1.0, 10.0)_SS2.0', 'PREDICT_original_phasef_monogenic_peak_position_WL3_N5', 'PREDICT_original_tf_Gabor_peak_F0.5_A2.36', 'PREDICT_original_tf_GLCM_dissimilarityd3.0A0.0', 'PREDICT_original_tf_GLCM_energyd3.0A2.36', 'PREDICT_original_tf_GLCMMS_energyd1.0A0.79std', 'PREDICT_original_tf_Gabor_range_F0.2_A0.0', 'PREDICT_original_tf_GLCMMS_contrastd1.0A1.57mean', 'PREDICT_original_tf_GLCMMS_ASMd1.0A1.57mean', 'PREDICT_original_tf_Gabor_quartile_range_F0.5_A1.57', 'PREDICT_original_vf_Frangi_full_mean_SR(1.0, 10.0)_SS2.0', 'PREDICT_original_tf_GLCMMS_energyd1.0A0.0mean', 'PREDICT_original_tf_GLCMMS_dissimilarityd1.0A2.36mean', 'PREDICT_original_vf_Frangi_edge_skewness_SR(1.0, 10.0)_SS2.0', 'PREDICT_original_tf_LBP_range_R8_P24', 'PREDICT_original_tf_GLCM_homogeneityd1.0A2.36', 'PREDICT_original_tf_GLCMMS_homogeneityd1.0A0.79std', 'PREDICT_original_tf_GLCMMS_contrastd3.0A0.79std', 'PREDICT_original_tf_GLCMMS_

In [40]:
### Training the classifiers
final_KNN_model = KNN_classifier(training_data_selected_df, label_training_binary)
final_RF_model = random_forest_classifier(training_data_selected_df, label_training_binary)
final_SVM_model = SVM_classifier(training_data_selected_df, label_training_binary)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


c:\Users\evade\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan nan nan nan nan nan nan nan]
scores:[nan nan nan 

KeyboardInterrupt: 

In [ ]:
#accuracy of the prediction of the test set
y_test = label_test_binary.values.flatten()
X_test = data_test_normalized_df.reset_index(drop=True)[training_data_selected_df.columns]

y_pred_KNN = final_KNN_model.predict(X_test)
test_accuracy_KNN = accuracy_score(y_test, y_pred_KNN)
print(f"Test Accuracy of KNN: {test_accuracy_KNN:.4f}")

y_pred_RF = final_RF_model.predict(X_test)
test_accuracy_RF = accuracy_score(y_test, y_pred_RF)
print(f"Test Accuracy of RF: {test_accuracy_RF:.4f}")

y_pred_SVM = final_SVM_model.predict(X_test)
test_accuracy_SVM = accuracy_score(y_test, y_pred_SVM)
print(f"Test Accuracy of SVM: {test_accuracy_SVM:.4f}")

Test Accuracy of RF: 0.7297
Test Accuracy of SVM: 0.8378


c:\Users\evade\anaconda3\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
